In [1]:
import numpy as np

def bermuda_option_pricing_binomial(
    S: float,      # 標的資產當前價格
    X: float,      # 履約價
    r: float,      # 無風險利率 (年化)
    sigma: float,  # 波動率 (年化)
    T: float,      # 到期時間 (年)
    n_steps: int,  # 二項式樹的總步數
    n_exec_dates: int, # 提前執行日期數量
    option_type: str # 'call' or 'put'
):
    """
    使用二項式樹模型評價百慕達型選擇權。
    """

    dt = T / n_steps  # 每個時間步長
    u = np.exp(sigma * np.sqrt(dt)) # 上漲因子
    d = 1 / u # 下跌因子
    p = (np.exp(r * dt) - d) / (u - d) # 上漲機率
    discount_factor = np.exp(-r * dt)

    # 建立股票價格樹
    stock_prices = np.zeros((n_steps + 1, n_steps + 1))
    stock_prices[0, 0] = S
    for i in range(1, n_steps + 1):
        stock_prices[i, 0] = stock_prices[i-1, 0] * u
        for j in range(1, i + 1):
            stock_prices[i, j] = stock_prices[i-1, j-1] * d

    # 初始化選擇權價值樹
    option_values = np.zeros((n_steps + 1, n_steps + 1))

    # 在到期日計算最終支付
    if option_type == 'call':  # S-K
        option_values[n_steps, :] = np.maximum(stock_prices[n_steps, :] - X, 0)
    else: # 'put'K-S
        option_values[n_steps, :] = np.maximum(X - stock_prices[n_steps, :], 0)

    # 從後往前倒推計算
    for i in range(n_steps - 1, -1, -1):
        for j in range(i + 1):
            # 計算選擇權的期望價值 (繼續持有)
            # 這是下一時間步的價值折現
            continue_value = (p * option_values[i+1, j] + (1-p) * option_values[i+1, j+1]) * discount_factor
            
            # 計算提前執行的內在價值
            exercise_value = np.maximum(stock_prices[i, j] - X, 0) if option_type == 'call' else np.maximum(X - stock_prices[i, j], 0)
            
            # 判斷是否為提前執行日
            # 這裡假設提前執行日是均勻分佈的
            is_early_exercise_date = (i + 1) % (n_steps // n_exec_dates) == 0

            if is_early_exercise_date:
                # 提前執行日，選擇繼續持有或提前執行中價值較高的
                option_values[i, j] = np.maximum(continue_value, exercise_value)
            else:
                # 非提前執行日，只能選擇繼續持有
                option_values[i, j] = continue_value
                
    return option_values[0, 0]

# --- 範例用法 ---

if __name__ == "__main__":
    # 輸入參數
    S = 100.0        # 標的價格
    X = 100.0        # 履約價
    r = 0.05         # 無風險利率 (5%)
    sigma = 0.20     # 波動率 (20%)
    T = 1.0          # 到期時間 (1年)
    n_steps = 100    # 二項式樹的步數
    n_exec_dates = 4 # 允許執行 4 次 (例如每季一次)
    
    # 計算百慕達型買權價格
    bermuda_call_price = bermuda_option_pricing_binomial(
        S, X, r, sigma, T, n_steps, n_exec_dates, 'call'
    )
    print(f"百慕達型買權價格: {bermuda_call_price:.4f}")

    # 計算百慕達型賣權價格
    bermuda_put_price = bermuda_option_pricing_binomial(
        S, X, r, sigma, T, n_steps, n_exec_dates, 'put'
    )
    print(f"百慕達型賣權價格: {bermuda_put_price:.4f}")

百慕達型買權價格: 10.4306
百慕達型賣權價格: 5.9549
